In [6]:
import neuroproof
import neuroproof.FocusedProofreading as fp
import numpy as np
import sys
sys.path.append('../')
import gp

In [2]:
graph = fp.Graph('/home/d/FP/dojo/graph_relabeled.json')

In [35]:
  def generate_M(graph, rhoana, min_pixels=100, max=1000, ignore_zero_neighbor=True):
    '''
    '''

    hist = gp.Util.get_histogram(rhoana.astype(np.uint64))
    labels = len(hist)

    batch_count = 0
    
    M = np.zeros((labels,labels), dtype=np.float32)

    for l in range(labels):

      if l == 0:
        continue

      if hist[l] < min_pixels:
        continue

#       print l
        
      neighbors = sorted(gp.Util.grab_neighbors(rhoana, l))

      graph_neighbors = sorted(graph.find_close_bodies(l,0,0.))

      pred = [ne[1] for ne in graph_neighbors if ne[0] in neighbors]
        
    
      for i,n in enumerate(neighbors):
            
        if ignore_zero_neighbor and n == 0:
          continue

        if hist[n] < min_pixels:
          continue

        
        M[l,n] = pred[i]
        M[n,l] = pred[i]

    return M

In [29]:
image, prov, gold, rhoana, bb = gp.Legacy.read_dojo_data()

a


In [36]:
bigM = []
for z in range(rhoana.shape[0]):
    M = generate_M(graph, rhoana[z])
    bigM.append(M)
    print 'stored z=',z

stored z= 0
stored z= 1
stored z= 2
stored z= 3
stored z= 4
stored z= 5
stored z= 6
stored z= 7
stored z= 8
stored z= 9


In [33]:
import cPickle as pickle

In [37]:
with open('/home/d/dojo_xp/data/bigM_fp.p', 'wb') as f:
    pickle.dump(bigM, f)